In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
data = pd.read_csv(r"D:\Intellipaat\kegal datasets\steel plates dataset\train.csv")
data

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19214,19214,749,757,143210,143219,17,4,4,2193,122,...,-0.1429,0.0044,0.2901,0,0,0,1,0,0,0
19215,19215,723,735,2488529,2488541,231,17,26,27135,104,...,0.7222,-0.0989,0.5378,0,0,0,0,0,0,1
19216,19216,6,31,1578055,1578129,780,114,98,71112,41,...,0.7719,-0.4283,0.9997,1,0,0,0,0,0,0
19217,19217,9,18,1713172,1713184,126,13,26,14808,88,...,0.9610,-0.1162,0.3509,0,0,0,0,0,0,1


In [3]:
col_name = []
with open(r"D:\Intellipaat\kegal datasets\steel plates dataset\Faults27x7_var") as f:
    for line in f:
        col_name.append(line.strip())

In [4]:
data  = data.drop(columns=['id'])

#### for now don't use original data, just use train.csv

In [5]:
org = pd.read_csv("D:\Intellipaat\kegal datasets\steel plates dataset\Faults.NNA",engine= "python",delimiter= '\s',names= col_name)
org

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [6]:
df = pd.concat([data,org],ignore_index= True)
df

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21155,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
21156,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
21157,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
21158,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [7]:
X = df.iloc[:,:-7]

In [8]:
pastry = df['Pastry'].copy()
z_scratch = df['Z_Scratch'].copy()
k_scatch = df['K_Scatch'].copy()
stains = df['Stains'].copy()
dirtiness = df['Dirtiness'].copy()
bumps = df['Bumps'].copy()
other_faults = df['Other_Faults'].copy()

In [9]:
Y = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
Y_col_name = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

In [10]:
y_params_dict = {}
for y_nm in Y_col_name:
    y_params_dict[y_nm] = None

In [11]:
y_scores = []

In [12]:
cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for y,yn in zip(Y,Y_col_name):
    
    print(f'trial: {yn} \n')
    
    def objective_xgbc(trial):
    
        cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        params = dict(
            n_estimators = trial.suggest_int('n_estimators',100,500),
            max_depth = trial.suggest_int('max_depth',2,64),
            # here it has to be max_leaves instead of num_leaves at both places
            num_leaves = trial.suggest_int('num_leaves',2,128),
            learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
            # no min_child_samples in xgbc
            # and gamma is missing
            min_child_samples = trial.suggest_int('min_child_samples',2,500),
            min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
            subsample = trial.suggest_float('subsample', 0.33,0.85),
            colsample_bytree = trial.suggest_float('colsample_bylevel',0.33,0.7),
            reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
            reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
        )
        
        xgbc = XGBClassifier(random_state= 0, objective='binary:logistic',**params)
        
        pipe = Pipeline(
            steps = [
                ('scaler', StandardScaler()),
                ('model', xgbc)
            ]
        )
        
        score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
        return score
    
    study_xgbc = optuna.create_study(direction='maximize')
    study_xgbc.optimize(objective_xgbc, n_trials=1000, n_jobs=-1, show_progress_bar=True)
    
    best_params = study_xgbc.best_params
    best_score = study_xgbc.best_value
    
    time.sleep(5)
    clear_output()
    
    
    y_scores.append((yn,f'score: {best_score}'))
    
    with open('xgbc_results.txt', 'w') as file:
        for tup in y_scores:
            file.write(f'{tup[0]}, {tup[1]} \n')

In [13]:
best_score

0.725214095041119

In [14]:
y_scores

[('pastry', 'score: 0.8819649892209922'),
 ('z_scratch', 'score: 0.9668366040634366'),
 ('k_scatch', 'score: 0.986865731713498'),
 ('stains', 'score: 0.9936102735136453'),
 ('dirtiness', 'score: 0.9058528217839565'),
 ('bumps', 'score: 0.8232360850907245'),
 ('other_faults', 'score: 0.725214095041119')]

In [19]:
best_params

{'n_estimators': 392,
 'max_depth': 6,
 'num_leaves': 50,
 'learning_rate': 0.016645715817881778,
 'min_child_samples': 164,
 'min_child_weight': 6.094805630141306,
 'subsample': 0.8003137441528139,
 'colsample_bylevel': 0.42330584333222626,
 'reg_alpha': 0.06219727110848896,
 'reg_lambda': 0.07211771372512957}

In [21]:
test_org = pd.read_csv(r"D:\Intellipaat\kegal datasets\steel plates dataset\test.csv")
test_org

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,19219,1015,1033,3826564,3826588,659,23,46,62357,67,...,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,19220,1257,1271,419960,419973,370,26,28,39293,92,...,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,19221,1358,1372,117715,117724,289,36,32,29386,101,...,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,19222,158,168,232415,232440,80,10,11,8586,107,...,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,19223,559,592,544375,544389,140,19,15,15524,103,...,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12809,32028,1101,1116,447943,447992,313,32,37,21603,79,...,0.0126,0.4063,0.9194,1.0,2.4955,1.2305,1.6335,0.7661,-0.3109,0.8894
12810,32029,1289,1306,3149494,3149542,59,9,18,5249,113,...,0.0052,0.7778,1.0000,1.0,1.7708,0.8451,1.2553,0.7222,-0.0448,0.1954
12811,32030,41,210,1587535,1587191,16584,796,522,1858162,24,...,0.1236,0.2199,0.4097,0.0,4.2525,2.2504,2.2672,-0.0629,-0.0801,1.0000
12812,32031,1329,1340,702237,702267,386,43,34,36875,66,...,0.0095,0.2407,1.0000,1.0,2.5866,1.1139,1.5911,0.8461,-0.2629,0.7844


In [25]:
pastry_params = y_params_dict['pastry']
pastry_params

In [23]:
pastry_params = y_params_dict['pastry']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

TypeError: xgboost.sklearn.XGBClassifier() argument after ** must be a mapping, not NoneType

In [ ]:
pastry_params = y_params_dict['z_scratch']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [ ]:
pastry_params = y_params_dict['k_scatch']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [ ]:
pastry_params = y_params_dict['stains']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [ ]:
pastry_params = y_params_dict['dirtiness']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [ ]:
pastry_params = y_params_dict['bumps']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [ ]:
pastry_params = y_params_dict['other_faults']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]